In [53]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import pandas as pd

data = pd.read_excel("export_dataframe_translated.xlsx")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36710 entries, 0 to 36709
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           36710 non-null  datetime64[ns]
 1   id             36710 non-null  int64         
 2   content        36710 non-null  object        
 3   username       36710 non-null  object        
 4   like_count     36710 non-null  int64         
 5   retweet_count  36710 non-null  int64         
 6   language       36710 non-null  object        
 7   game           36710 non-null  object        
 8   opinion        358 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 2.5+ MB


In [54]:
data = data.head(356)
display(data)

,date,id,content,username,like_count,retweet_count,language,game,opinion
0,2023-04-07 23:51:58,1644488216316133377,playing vs yur soon 🫡,MarkeCSGO,231,4,en,CSGO,0.0
1,2023-04-07 23:49:18,1644487545361928192,you are more likely to get busted for jaywalki...,urnotjustin,31,3,en,CSGO,1.0
2,2023-04-07 23:45:26,1644486571708563458,looking for +2 support / anchor players min re...,unadaptiveCSGO,29,12,en,CSGO,0.0
3,2023-04-07 23:44:55,1644486443597721600,can you recognize this ak-47? just name the sk...,r1_skins,181,182,en,CSGO,0.0
4,2023-04-07 23:36:00,1644484196642037762,They never participated in a major in this pro...,paiNGamingCSGO,18,0,pt,CSGO,0.0
...,...,...,...,...,...,...,...,...,...
351,2023-04-06 23:49:37,1644125238488100865,"🔴Notic! : It has been found that the ""Slime"" U...",NoticiasUgc,156,10,es,Roblox,0.0
352,2023-04-06 23:46:05,1644124346804207617,Free Limited UGC Item: A slime was placed for ...,Roblox_RTC_BR,169,10,pt,Roblox,0.0
353,2023-04-06 23:43:39,1644123736080105476,💰 50 robux giveaway! how to enter. 1️⃣follow 2...,JoJewyd,52,38,en,Roblox,0.0
354,2023-04-06 23:43:18,1644123648758890496,released a gold helmet limited! very small sto...,etsulo,18,3,en,Roblox,0.0


In [55]:
comments = data['content'] 
labels = data['opinion']

vocab_size = 40000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(comments)
word_index = tokenizer.word_index


sequences = tokenizer.texts_to_sequences(comments)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(comments)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)


In [46]:
comments

0                                 playing vs yur soon 🫡 
1      you are more likely to get busted for jaywalki...
2      looking for +2 support / anchor players min re...
3      can you recognize this ak-47? just name the sk...
4      They never participated in a major in this pro...
                             ...                        
352    Free Limited UGC Item: A slime was placed for ...
353    💰 50 robux giveaway! how to enter. 1️⃣follow 2...
354    released a gold helmet limited! very small sto...
355                          repost to scare doors fans 
356    i have never seen a limited sell out that fast...
Name: content, Length: 357, dtype: object

In [47]:
print(testing_padded)

[[   0    0    0 ...  817   86  241]
 [   0    0    0 ...  820   14   37]
 [   0    0    0 ...   22   39  328]
 ...
 [   0    0    0 ...   39   12   53]
 [   0    0    0 ... 2430  809  589]
 [   0    0    0 ...   12   25  109]]


In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, shuffle=True)

In [78]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [79]:
# Kompilowanie modelu 
import numpy as np
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Trenowanie modelu 
history = model.fit(X_train, y_train, epochs=20, validation_data=(testing_padded, labels))

Epoch 1/20
9/9 [==============================] - 1s 20ms/step - loss: 0.6820 - accuracy: 0.7324 - val_loss: 0.6738 - val_accuracy: 0.7472
Epoch 2/20
9/9 [==============================] - 0s 9ms/step - loss: 0.6696 - accuracy: 0.7324 - val_loss: 0.6606 - val_accuracy: 0.7472
Epoch 3/20
9/9 [==============================] - 0s 9ms/step - loss: 0.6570 - accuracy: 0.7324 - val_loss: 0.6477 - val_accuracy: 0.7472
Epoch 4/20
9/9 [==============================] - 0s 9ms/step - loss: 0.6453 - accuracy: 0.7324 - val_loss: 0.6350 - val_accuracy: 0.7472
Epoch 5/20
9/9 [==============================] - 0s 9ms/step - loss: 0.6338 - accuracy: 0.7324 - val_loss: 0.6227 - val_accuracy: 0.7472
Epoch 6/20
9/9 [==============================] - 0s 9ms/step - loss: 0.6226 - accuracy: 0.7324 - val_loss: 0.6111 - val_accuracy: 0.7472
Epoch 7/20
9/9 [==============================] - 0s 10ms/step - loss: 0.6122 - accuracy: 0.7324 - val_loss: 0.6005 - val_accuracy: 0.7472
Epoch 8/20
9/9 [================

In [80]:
y_test = list(y_test)
y_test = [int(x) for x in y_test]

In [81]:
predictions = model.predict(X_test)
predicted_labels = [1 if prediction > 0.5 else 0 for prediction in predictions]
suma = 0
for i in range(len(predicted_labels)):
    if predicted_labels[i] == y_test[i]:
        suma += 1

print(suma/len(predicted_labels))

3/3 [==============================] - 0s 2ms/step
0.8055555555555556
